In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
# wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
#                            rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
gscene.show_pose(crob.home_pose)

Please create a subscriber to the marker


## Door

In [4]:
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [5]:
##
# @class KnobTask
# @brief sweep action points in alphabetical order
# @remark   state_param: boolean vector of which each element represents if each waypoint is covered or not
#           node_item: number of covered waypoints
class KnobTask(HingeTask):
    def __init__(self, oname, geometry, binding_pairs, knob_plug, sub_binders_dict=None, tol=1e-3):
        self.knob_plug = knob_plug
        HingeTask.__init__(self, oname, geometry, binding_pairs, sub_binders_dict=sub_binders_dict, tol=tol)
    ##
    # @brief (prototype) set state param
    # @param binding BindingTransform
    # @param state_param
    def set_state(self, binding, state_param):
        if self.knob_plug is not None:
            if binding.chain.handle_name == self.action_points_order[-1]:
                self.knob_plug.available=True
            else:
                self.knob_plug.available=False
        HingeTask.set_state(self, binding, state_param)

In [6]:
def add_lever(pscene, knob, door_s=None, lname="lever", lever_ang=np.pi / 6, knob_offset=(0.09), dims=(0.02, 0.2, 0.02),
              link_name="base_link", color=(0.8, 0.8, 0.8, 1), clearance=1e-3):
    gscene = pscene.gscene
    Q0 = [0] * gscene.joint_num
    Tbk = knob.get_tf(Q0)
    Tbkp = np.matmul(Tbk, SE3(np.identity(3), (0, 0, knob.dims[2] / 2 + clearance)))
    Tbl = np.matmul(Tbkp, SE3(np.identity(3), (0, knob_offset, dims[2] / 2)))
    lever = gscene.create_safe(GEOTYPE.BOX, lname, link_name=link_name,
                               center=Tbl[:3, 3], rpy=Rot2rpy(Tbl[:3, :3]), dims=dims,
                               fixed=False, collision=True, color=color)
    lgrasp = Grasp2Point("gp", lever, (0, 0, 0), (np.pi / 2, 0, -np.pi / 2))
    lattach = FramePoint("cp", lever, (0, -knob_offset, -dims[2] / 2), (0, 0, 0), key=lname)

    lever_plug = pscene.create_binder(bname=lname + "_plug", gname=lname, _type=KnobFramer,
                                      point=(0, -knob_offset, -dims[2] / 2), key=knob.name)
        
    if door_s is not None:
        knob_plug = pscene.create_binder(bname=knob.name+"_plug", gname=lname, 
                                         _type=FramedTool, point=(0, -knob_offset, -dims[2] / 2), rpy=(0, 0, 0), key=knob.name)
        knob_plug.available = False
        door_s.action_points_dict[knob.name] = FramePoint(knob.name, knob, (0, 0, knob.dims[2] / 2 + clearance), (0, 0, lever_ang), key=knob.name)
        
    lever_s = pscene.create_subject(oname=lname, gname=lname, _type=CustomObject,
                                    action_points_dict={lgrasp.name: lgrasp, lattach.name: lattach},
                                    sub_binders_dict={lever_plug.name: lever_plug, 
                                                      knob_plug.name: knob_plug})
    
    bd1 = (KnobFrame("r1", knob, (0, 0, knob.dims[2] / 2 + clearance), (0, 0, 0), key=knob.name),
           pscene.create_binder(bname=knob.name + "_1", gname=knob.name, _type=AttachFramer,
                                point=(0, 0, knob.dims[2] / 2 + clearance), key=lname))
    bd2 = (KnobFrame("r2", knob, (0, 0, knob.dims[2] / 2 + clearance), rpy=(0, 0, lever_ang), key=knob.name),
           pscene.create_binder(bname=knob.name + "_2", gname=knob.name, _type=AttachFramer,
                                point=(0, 0, knob.dims[2] / 2 + clearance), rpy=(0, 0, lever_ang), key=lname)
           )

    
    knob_s = pscene.create_subject(oname=knob.name, gname=knob.name, _type=KnobTask,
                                   binding_pairs=[bd1, bd2], knob_plug=knob_plug)
    return lever_s, knob_s

In [7]:
def add_door(pscene, dname = "door", center = (0.5,0,0.5), rpy = (0,0,0),
             dims = (0.05, 0.4, 0.5), hinge_point = (-0.025, 0.2, 0),
             door_ang = np.pi*3/4, door_div = 3, link_name="base_link", clearance=1e-3,
             frame_depth=0.0, frame_thickness=0.01, color = (0.8,0.8,0.8,1), add_frame=False):
    gscene = pscene.gscene
    door_frame = gscene.create_safe(GEOTYPE.BOX, dname+"_frame", link_name=link_name,
                                    center=center, rpy=Rot2rpy(np.matmul(Rot_rpy(rpy), Rot_axis(2,np.pi/2))),
                                    dims=np.asarray(dims)[[2,1,0]],
                                    fixed=True, collision=False, color=(1,0,0,0.1), display=True)
    door = gscene.create_safe(GEOTYPE.BOX, dname, link_name=link_name,
                              center=center, rpy=rpy, dims=dims,
                              fixed=False, collision=True, color=color)
    if add_frame:
        gscene.add_virtual_guardrail(door_frame, axis="xy", color=color, THICKNESS=frame_thickness,
                                     HEIGHT=dims[0]/2+frame_depth, margin=frame_thickness/2+clearance)

    hinge_p = FramePoint("hinge", door, hinge_point, (0,0,0), key=dname)
    
    door_hinge = pscene.create_binder(bname=door.name+"_hinge", gname=door.name, _type=HingeFramer,
                                  point=hinge_point, key=dname)

    door_s = pscene.create_subject(oname=dname, gname=dname, _type=CustomObject,
                                   action_points_dict={hinge_p.name: hinge_p},
                                   sub_binders_dict={door_hinge.name: door_hinge})

    hinge_bindings = []
    for i_div in range(door_div+1):
        ang = door_ang/door_div*i_div
        hp = HingeFrame("h{}".format(i_div), door_frame,
                        np.asarray(hinge_point)[[2,1,0]],
                        Rot2rpy(np.matmul(Rot_axis(2,np.pi/2).transpose(),
                                          Rot_axis(3, -ang))),
                        key=dname)
        bp =pscene.create_binder(bname=door_frame.name + "_{}".format(i_div), 
                                 gname=door_frame.name, _type=AttachFramer,
                                 point=np.asarray(hinge_point)[[2,1,0]], 
                                 rpy=Rot2rpy(np.matmul(Rot_axis(2,np.pi/2).transpose(),
                                                       Rot_axis(3, -ang))), 
                                 key=dname)
        hinge_bindings.append((hp, bp))

    hinge_s = pscene.create_subject(oname=door_frame.name, gname=door_frame.name, _type=HingeTask,
                                  binding_pairs=hinge_bindings)
    return door_s, hinge_s

In [8]:
door_s, hinge_s = add_door(pscene, center=(0.7,-0.2,0.5), color=(0.8,0.4,0.2,1))
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(-door_s.geometry.dims[0]/2-0.01-0.001,-door_s.geometry.dims[1]/2+0.05,0), 
                          rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=False, collision=True, color=(0.8,0.8,0.8,1), parent="door")

lever_s, knob_s = add_lever(pscene, knob, door_s, dims=(0.02,0.1,0.02), knob_offset=0.04)

### prepare planner

In [9]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [10]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=30)
tplan.prepare()

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

   Use a production WSGI server instead.
 * Debug mode: off


In [11]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True
def explicit_rule(pscene, node, leaf):
    if node[-2:] == ("r2", "knob"):
            return False
    if node[-2] != leaf[-2] and node[-1] != "grip0":
            return False
    if node[-1]=="grip0" and leaf[-1]=="knob" and node[-2]!="r1":
            return False
    if node[1] != leaf[1] and node[0] != "lever":
            return False
    if node[0] == "lever" and node[-2] != leaf[-2]:
            return False
    if node[:2] != leaf[:2] and node[-2]!="r2":
            return False
    return True
tplan.explicit_rule = explicit_rule

### Plan

In [12]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['door', 'door_frame', 'knob', 'lever']
('door_frame', 'h0', 'r1', 'knob')


In [13]:
goal_nodes = [('door_frame', 'h1', 'r1', 'knob')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=True, add_homing=True)

Use 18/36 agents
list index out of range
try: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
error: 1.8
try transition motion
try: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
error: 2.0
try transition motion
transition motion tried: True
result: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0') = success
branching: 0->1 (0.29/100.0 s, steps/err: 109(151.974916458 ms)/0.00171051390143)
try: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
result: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
try: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
branching: 1->2 (0.32/100.0 s, steps/err: 1(4.72521781921 ms)/0)
try: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
error: 2.1
try transition motion
error: 2.1
try: 2 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0'

try: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
try: 11 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
try: 9 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
branching: 1->16 (0.79/100.0 s, steps/err: 1(28.685092926 ms)/0)
error: 0.73
error: 1.0
Motion Plan Failure
try transition motion
error: 0.73
joint max
result: 5 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = fail
list index out of range
result: 15 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
result: 5 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = fail
constrained motion tried: False
list index out of range
branching: 15->17 (0.81/100.0 s, steps/err: 1(9.87410545349 ms)/0)
try transition motion
try: 14 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
result: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1'

branching: 9->32 (1.04/100.0 s, steps/err: 9(148.018836975 ms)/0.00181100274626)
result: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0') = success
try: 26 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
try: 34 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
error: 1.9
result: 18 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0') = success
branching: 0->34 (1.05/100.0 s, steps/err: 63(721.071004868 ms)/0.0015741273672)
try: 26 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
try transition motion
try: 24 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
try: 32 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
result: 34 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
result: 26 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
branching: 25->

try: 46 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
error: 0.73
error: 0.072
try: 49 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
try transition motion
Motion Plan Failure
try: 49 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
joint min
constrained motion tried: True
list index out of range
constrained motion tried: False
error: 0.73
result: 34 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
result: 26 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
result: 49 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
branching: 34->52 (1.19/100.0 s, steps/err: 1(25.2380371094 ms)/0)
result: 49 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
result: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
Motion Plan Failure
branching: 49->54 (1.19/100

Motion Plan Failure
result: 58 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = fail
collision
branching: 24->68 (1.38/100.0 s, steps/err: 53(373.502969742 ms)/1.5334707687e-05)
constrained motion tried: False
result: 23 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
collision
branching: 23->67 (1.39/100.0 s, steps/err: 1(6.20198249817 ms)/0)
constrained motion tried: False
try: 49 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
try: 54 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
collision
constrained motion tried: False
Motion Plan Failure
error: 0.73
Motion Plan Failure
try: 44 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
Motion Plan Failure
result: 54 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
error: 0.65
result: 54 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = fail
branching: 54->69 (1.42/1

...

result: 54 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = fail
end
branching: 108->129 (2.27/100.0 s, steps/err: 53(308.315992355 ms)/1.01514601457e-05)
result: 113 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
constrained motion tried: True
try: 125 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0')
Motion Filer Failure: ReachChecker
collision
try: 119 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
constrained motion tried: False
try: 76 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
result: 125 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0') = fail
Motion Plan Failure
Motion Plan Failure
error: 2.7
result: 107 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
result: 104 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
error: 2.7
try: 54 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0')
c

try: 139 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
result: 128 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
result: 114 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
branching: 90->138 (2.57/100.0 s, steps/err: 80(451.869010925 ms)/0.000225680465694)
Motion Plan Failure
branching: 138->140 (2.57/100.0 s, steps/err: 1(6.69693946838 ms)/0)
try: 138 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
branching: 128->142 (2.57/100.0 s, steps/err: 53(331.680059433 ms)/1.54253992946e-05)
branching: 114->141 (2.57/100.0 s, steps/err: 1(14.4250392914 ms)/0)
result: 134 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
error: 2.1
try: 139 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
result: 139 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0') = success
try: 143 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 

error: 0.73
error: 1.1
try: 138 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
end
error: 1.1
collision
constrained motion tried: True
end
constrained motion tried: False
constrained motion tried: True
end
result: 135 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = success
Motion Plan Failure
branching: 135->152 (2.88/100.0 s, steps/err: 80(383.239984512 ms)/0.000225680465694)
result: 92 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = fail
constrained motion tried: True
collision
result: 90 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = success
try: 152 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
end
result: 143 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
constrained motion tried: True
branching: 143->154 (2.89/100.0 s, steps/err: 53(286.270141602 ms)/4.81098595427e-06)
constrained motion tried: False
branching: 90->153 (2.89/100.0 s, steps/err:

result: 165 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
error: 1.1
++ adding return motion to acquired answer ++
Goal reached
try: 170 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 165->167 (3.07/100.0 s, steps/err: 1(7.64608383179 ms)/0)
result: 168 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
end
try: 149 - ('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
constrained motion tried: True
++ adding return motion to acquired answer ++
branching: 168->171 (3.1/100.0 s, steps/err: 1(14.4109725952 ms)/0)
Goal reached
Goal reached
collision
result: 170 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
constrained motion tried: False
branching: 170->172 (3.11/100.0 s, steps/err: 1(8.3749294281 ms)/0)
result: 149 - ('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = success
result: 150 - ('door_frame', 'h1',

Goal reached
result: 192 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
end
======================= terminated 5: required answers acquired  (3.4/100.0) ===============================
branching: 192->194 (3.36/100.0 s, steps/err: 1(8.55398178101 ms)/0)
collision
constrained motion tried: False
++ adding return motion to acquired answer ++
constrained motion tried: True
Motion Plan Failure
result: 152 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
result: 152 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
branching: 152->195 (3.38/100.0 s, steps/err: 80(426.347017288 ms)/0.000317717999605)
======================= terminated 13: Stop called from other agent  (3.4/100.0) ===============================
======================= terminated 2: Stop called from other agent  (3.4/100.0) ===============================
collision
constrained motion tried: False
Motion Plan Failure
result: 151 - ('lever', 'h

In [14]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [15]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'knob')


### push back

In [16]:
term_state = snode_schedule[-1].state
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

['door', 'door_frame', 'knob', 'lever']
('door_frame', 'h1', 'r1', 'knob')


In [17]:
goal_nodes = [('door_frame', 'h0', 'r1', 'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
list index out of range
try: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
error: 2.5
try: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
try transition motion
error: 2.2
try transition motion
try: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
error: 2.5
try transition motion
transition motion tried: True
result: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
transition motion tried: True
branching: 0->1 (0.26/30.0 s, steps/err: 91(170.984983444 ms)/0.00175834914444)
try: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
branching: 0->2 (0.28/30.0 s, steps/err: 77(165.586948395 ms)/0.00132880756993)
try: 2 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 1 - ('door_frame', 'h1', 'r1', 'grip0')

result: 16 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
branching: 10->17 (0.67/30.0 s, steps/err: 3(161.573171616 ms)/0.00188733659879)
try transition motion
try: 17 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
try: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 16->18 (0.7/30.0 s, steps/err: 1(18.4991359711 ms)/0)
error: 0.73
try: 17 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
list index out of range
result: 17 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
result: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
try: 15 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
branching: 1->20 (0.73/30.0 s, steps/err: 1(19.7448730469 ms)/0)
error: 0.041
branching: 17->19 (0.75/30.0 s, steps/err: 1(12.3369693756 ms)/0)
try transition motion
try: 5 - ('door_fr

result: 27 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
try: 32 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
try: 35 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
transition motion tried: True
error: 0.73
try: 35 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
result: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0') = success
error: 1.1
result: 36 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
transition motion tried: True
result: 33 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
try: 37 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
branching: 27->34 (0.97/30.0 s, steps/err: 1(18.0459022522 ms)/0)
result: 19 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
Motion Filer Failure: ReachChecker
branching: 36->38 (0.98/30.0 

branching: 5->53 (1.16/30.0 s, steps/err: 1(7.47609138489 ms)/0)
error: 0.73
branching: 29->52 (1.16/30.0 s, steps/err: 2(84.4411849976 ms)/0.00179714238629)
end
try: 52 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
constrained motion tried: True
try: 51 - ('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
try: 35 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
try: 47 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
result: 52 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
result: 25 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
error: 1.1
result: 51 - ('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = success
branching: 52->54 (1.19/30.0 s, steps/err: 1(8.87608528137 ms)/0)
branching: 51->56 (1.19/30.0 s, steps/err: 1(8.33106040955 ms)/0)
error: 0.73
branching: 25->55 (1.19/30.0 s, steps/err: 53(290.0478

branching: 61->67 (1.43/30.0 s, steps/err: 1(7.39693641663 ms)/0)
end
constrained motion tried: True
error: 0.29
end
try transition motion
joint min
constrained motion tried: True
try: 35 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
Motion Filer Failure: ReachChecker
constrained motion tried: False
Motion Plan Failure
result: 35 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
result: 35 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0') = fail
joint min
branching: 35->68 (1.45/30.0 s, steps/err: 80(422.603130341 ms)/7.7284572029e-05)
result: 46 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = fail
end
try: 46 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
constrained motion tried: True
try: 68 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
transition motion tried: True
constrained motion tried: False
result: 51 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame'

error: 0.041
end
result: 70 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
try: 68 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
try transition motion
constrained motion tried: True
try: 35 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
result: 84 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
branching: 70->83 (1.6/30.0 s, steps/err: 1(6.63495063782 ms)/0)
try: 87 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 84->86 (1.62/30.0 s, steps/err: 1(11.0769271851 ms)/0)
result: 35 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0') = success
result: 26 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
end
result: 6 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
branching: 26->87 (1.63/30.0 s, steps/err: 53(272.639989853 ms)/1.49427330722e-05)
branching: 

result: 74 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
branching: 74->106 (1.85/30.0 s, steps/err: 53(269.21081543 ms)/5.72290878085e-06)
end
result: 74 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
end
constrained motion tried: True
end
try: 104 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
constrained motion tried: True
result: 81 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
try: 103 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
branching: 81->107 (1.86/30.0 s, steps/err: 53(284.419775009 ms)/9.63352356909e-06)
try: 105 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
error: 0.73
constrained motion tried: True
try: 106 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
result: 74 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success

Goal reached
result: 123 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
constrained motion tried: True
branching: 123->126 (2.07/30.0 s, steps/err: 1(7.6060295105 ms)/0)
result: 88 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
Goal reached
error: 0.73
Goal reached
result: 99 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
try: 128 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 99->130 (2.1/30.0 s, steps/err: 53(308.556079865 ms)/9.7195171983e-06)
end
result: 128 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
branching: 128->131 (2.11/30.0 s, steps/err: 1(7.3459148407 ms)/0)
branching: 88->128 (2.1/30.0 s, steps/err: 53(291.3210392 ms)/9.67874130868e-06)
try: 67 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
constrained motion tried: True
try: 70 - ('lever', 'h0', 'r2', 'grip

result: 76 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
result: 120 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
branching: 76->144 (2.29/30.0 s, steps/err: 53(283.478021622 ms)/2.57346685198e-05)
branching: 120->143 (2.29/30.0 s, steps/err: 53(271.13199234 ms)/5.72290878085e-06)
======================= terminated 12: Stop called from other agent  (2.3/30.0) ===============================
======================= terminated 9: required answers acquired  (2.3/30.0) ===============================
======================= terminated 2: Stop called from other agent  (2.3/30.0) ===============================
======================= terminated 17: Stop called from other agent  (2.3/30.0) ===============================
end
constrained motion tried: True
result: 76 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
branching: 76->145 (2.32/30.0 s, steps/err: 53(250.091075897 ms)/

In [18]:
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)

In [19]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'knob')
